> Created on Thu Jul  9 22:10:09 2020  @author: Richie Bao-caDesign设计(cadesign.cn)

## 1 核密度估计

In [6]:
import numpy as np
from scipy import stats
import math





In [7]:
x=np.linspace(stats.norm.ppf(0.01),stats.norm.ppf(0.99), 100)

In [8]:
x

array([-2.32634787, -2.27935095, -2.23235402, -2.18535709, -2.13836017,
       -2.09136324, -2.04436631, -1.99736939, -1.95037246, -1.90337553,
       -1.85637861, -1.80938168, -1.76238475, -1.71538783, -1.6683909 ,
       -1.62139397, -1.57439705, -1.52740012, -1.48040319, -1.43340627,
       -1.38640934, -1.33941241, -1.29241549, -1.24541856, -1.19842163,
       -1.15142471, -1.10442778, -1.05743085, -1.01043393, -0.963437  ,
       -0.91644007, -0.86944314, -0.82244622, -0.77544929, -0.72845236,
       -0.68145544, -0.63445851, -0.58746158, -0.54046466, -0.49346773,
       -0.4464708 , -0.39947388, -0.35247695, -0.30548002, -0.2584831 ,
       -0.21148617, -0.16448924, -0.11749232, -0.07049539, -0.02349846,
        0.02349846,  0.07049539,  0.11749232,  0.16448924,  0.21148617,
        0.2584831 ,  0.30548002,  0.35247695,  0.39947388,  0.4464708 ,
        0.49346773,  0.54046466,  0.58746158,  0.63445851,  0.68145544,
        0.72845236,  0.77544929,  0.82244622,  0.86944314,  0.91

In [9]:
stats.norm.ppf(0.01)

-2.3263478740408408

In [11]:
stats.norm.stats(moments='mvsk')

(array(0.), array(1.), array(0.), array(0.))

In [12]:
stats.norm.ppf(0.01)

-2.3263478740408408